In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("pyspark-shapefiles").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
from magellan.types import Point, Polygon
from pyspark.sql import Row
from pyspark.sql.functions import *

# Dataset

In [5]:
df = spark.read.format("magellan").load('/glade/u/home/abanihi/data/shapefiles/states/').cache()

In [6]:
df.show()

+-----+--------+--------------------+--------------------+-----+
|point|polyline|             polygon|            metadata|valid|
+-----+--------+--------------------+--------------------+-----+
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.Polygon@...|Map(STATE_ABBR ->...| true|
| null|    null|magellan.

In [7]:
df.printSchema()

root
 |-- point: point (nullable = true)
 |-- polyline: polyline (nullable = true)
 |-- polygon: polygon (nullable = true)
 |-- metadata: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- valid: boolean (nullable = true)



**select the polygon and metadata columns and get rid of the other columns since they are not useful in this case.**

In [9]:
states_df = df.select(df['metadata'], df['polygon'])
states_df.show()

+--------------------+--------------------+
|            metadata|             polygon|
+--------------------+--------------------+
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.Polygon@...|
|Map(STATE_ABBR ->...|magellan.P

**Get state abbreviation from metadata column and append a new column with states abbreviation to the dataframe.**

In [10]:
states_df = states_df.withColumn("states", df['metadata']['STATE_ABBR'])

In [11]:
states_df.show()

+--------------------+--------------------+------+
|            metadata|             polygon|states|
+--------------------+--------------------+------+
|Map(STATE_ABBR ->...|magellan.Polygon@...|    DC|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    WY|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    MA|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    VT|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    SC|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    MO|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    AR|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    UT|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    MT|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    CT|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    MS|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    GA|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    PA|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    MN|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    NH|
|Map(STATE_ABBR ->...|magellan.Polygon@...|    IN|
|Map(STATE_ABBR ->...|magellan.

In [32]:
a = Point(40.0, 100.0)
a

Point (40.0,100.0)

In [36]:
b = a.convert()
type(b)

shapely.geometry.point.Point

In [31]:
states_df.select(Point(40.0, 100.0).within(states_df["polygon"]))

AttributeError: 'Point' object has no attribute 'within'

In [21]:
polygons = states_df.select("polygon")#.flatMap(lambda x: x).collect()

In [23]:
b = polygons.rdd

In [24]:
b

MapPartitionsRDD[33] at javaToPython at NativeMethodAccessorImpl.java:0

In [39]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [65]:
contains_udf = udf(lambda x: True if a.toShapely().within(x.toShapely()) else False, BooleanType())

In [66]:
m = states_df.withColumn("contains", contains_udf(states_df['polygon']))

In [69]:
m.describe().show()

+-------+------+
|summary|states|
+-------+------+
|  count|    49|
|   mean|  null|
| stddev|  null|
|    min|    AL|
|    max|    WY|
+-------+------+



In [70]:
m.printSchema()

root
 |-- metadata: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- polygon: polygon (nullable = true)
 |-- states: string (nullable = true)
 |-- contains: boolean (nullable = true)



In [72]:
m.select(m["contains"]).show()

Py4JJavaError: An error occurred while calling o1503336.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 4 times, most recent failure: Lost task 0.3 in stage 39.0 (TID 30, 10.148.2.150, executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/glade/p/work/abanihi/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/glade/p/work/abanihi/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/glade/p/work/abanihi/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 106, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "/glade/p/work/abanihi/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 144, in load_stream
    yield self._read_with_length(stream)
  File "/glad